## Business Context
Danish grocery retailers run weekly promotions.
Poor demand forecasting during promotions leads to stockouts,
lost revenue, and increased food waste.

## Core Business Question
Which promotions cause stockouts, where do they occur,
and how much revenue is lost due to forecasting gaps?


## Data Grain

Only weeks with promotion_flag = 1 are considered promotion weeks for KPI calculations. 

One row per Store × Product × Promotion Week

## KPIs
1. Promotion Stockout Rate (%)
2. Lost Revenue (DKK)
3. Forecast Error (MAPE)
...

## Synthetic Data Generation
This section generates realistic Danish retail promotion data
to analyze stockouts, lost revenue, and forecasting gaps.

In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)


In [2]:
regions = ["Sjælland", "Jylland"]

stores = [f"Store_{i:02d}" for i in range(1, 26)]  # 25 stores

product_categories = [
    "Dairy", "Meat", "Produce", "Bakery", "Frozen", "Dry Goods"
]

weeks = list(range(1, 17))  # 16 promotion weeks


In [3]:
data = []

for week in weeks:
    for store in stores:
        region = np.random.choice(regions, p=[0.55, 0.45])
        for category in product_categories:
            
            promotion_flag = np.random.choice([0, 1], p=[0.4, 0.6])
            
            forecast_units = np.random.randint(80, 200)
            
            demand_multiplier = np.random.normal(1.0, 0.25)
            actual_demand = int(forecast_units * demand_multiplier)
            actual_units = max(min(actual_demand, forecast_units * 2), 0)
            
            stockout_hours = 0
            if promotion_flag == 1 and actual_units > forecast_units:
                stockout_hours = np.random.randint(2, 24)
            
            unit_price = np.random.uniform(10, 40)
            ending_inventory = max(forecast_units - actual_units, 0)

            data.append([
                week,
                region,
                store,
                category,
                promotion_flag,
                forecast_units,
                actual_units,
                stockout_hours,
                round(unit_price, 2),
                ending_inventory
            ])


In [4]:
columns = [
    "week",
    "region",
    "store_id",
    "product_category",
    "promotion_flag",
    "forecast_units",
    "actual_units",
    "stockout_hours",
    "unit_price_dkk",
    "ending_inventory"
]

df = pd.DataFrame(data, columns=columns)

df.head()


,week,region,store_id,product_category,promotion_flag,forecast_units,actual_units,stockout_hours,unit_price_dkk,ending_inventory
0,1,Sjælland,Store_01,Dairy,1,186,208,20,13.00,0
1,1,Sjælland,Store_01,Meat,1,196,263,5,14.29,0
2,1,Sjælland,Store_01,Produce,1,132,112,0,15.50,20
3,1,Sjælland,Store_01,Bakery,0,101,87,0,10.21,14
4,1,Sjælland,Store_01,Frozen,0,170,126,0,18.76,44


In [5]:
df.describe(include="all")

,week,region,store_id,product_category,promotion_flag,forecast_units,actual_units,stockout_hours,unit_price_dkk,ending_inventory
count,2400.000000,2400,2400,2400,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000,2400.000000
unique,NaN,2,25,6,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Sjælland,Store_01,Dairy,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1380,96,400,NaN,NaN,NaN,NaN,NaN,NaN
mean,8.500000,NaN,NaN,NaN,0.603333,139.119167,138.604167,3.873333,24.941562,14.604583
std,4.610733,NaN,NaN,NaN,0.489308,34.552025,50.312670,6.863630,8.583106,22.147651
min,1.000000,NaN,NaN,NaN,0.000000,80.000000,17.000000,0.000000,10.010000,0.000000
25%,4.750000,NaN,NaN,NaN,0.000000,109.000000,100.750000,0.000000,17.647500,0.000000
50%,8.500000,NaN,NaN,NaN,1.000000,139.000000,132.000000,0.000000,24.910000,0.000000
75%,12.250000,NaN,NaN,NaN,1.000000,169.000000,172.000000,6.000000,32.330000,24.000000


In [6]:
df.to_csv("promotion_performance.csv", index=False)

In [7]:
df.shape

(2400, 10)

In [8]:
df[df["stockout_hours"] > 0].head()

,week,region,store_id,product_category,promotion_flag,forecast_units,actual_units,stockout_hours,unit_price_dkk,ending_inventory
0,1,Sjælland,Store_01,Dairy,1,186,208,20,13.00,0
1,1,Sjælland,Store_01,Meat,1,196,263,5,14.29,0
9,1,Jylland,Store_02,Bakery,1,190,235,8,28.30,0
18,1,Jylland,Store_04,Dairy,1,84,88,8,35.50,0
19,1,Jylland,Store_04,Meat,1,112,128,17,21.12,0


In [9]:
df.groupby("promotion_flag")["stockout_hours"].mean()

promotion_flag
0    0.00000
1    6.41989
Name: stockout_hours, dtype: float64